In [ ]:
!pip install gradio tensorflow numpy requests


Gradio


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the pre-trained model
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Add a new output layer
num_classes = 24  # replace with the number of Indian monuments you have
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

# Create the new model
model = tf.keras.models.Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the image generators for training and testing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Define the paths to your train and test data sets
train_dir = '/content/drive/MyDrive/images/train'
test_dir = '/content/drive/MyDrive/images/test'

# Set the batch size
batch_size = 32

# Create the image generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # replace with the number of epochs you want to train for
    steps_per_epoch=len(train_generator),
    validation_data=test_generator,
    validation_steps=len(test_generator)
)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

# Save the model
model.save('/content/drive/MyDrive/images/IMR.h5')


94668760/94668760 [==============================] - 4s 0us/step
Found 3672 images belonging to 24 classes.
Found 1058 images belonging to 24 classes.
Epoch 1/10
115/115 [==============================] - 1804s 16s/step - loss: 1.4111 - accuracy: 0.6296 - val_loss: 2.9807 - val_accuracy: 0.4887
Epoch 2/10
115/115 [==============================] - 477s 4s/step - loss: 0.5119 - accuracy: 0.8794 - val_loss: 3.4093 - val_accuracy: 0.5009
Epoch 3/10
115/115 [==============================] - 481s 4s/step - loss: 0.3530 - accuracy: 0.9120 - val_loss: 3.7778 - val_accuracy: 0.5161
Epoch 4/10
115/115 [==============================] - 426s 4s/step - loss: 0.2795 - accuracy: 0.9325 - val_loss: 4.0266 - val_accuracy: 0.5057
Epoch 5/10
115/115 [==============================] - 474s 4s/step - loss: 0.2243 - accuracy: 0.9496 - val_loss: 4.1935 - val_accuracy: 0.5198
Epoch 6/10
115/115 [==============================] - 476s 4s/step - loss: 0.1931 - accuracy: 0.9559 - val_loss: 4.4773 - val_accura

In [ ]:
!pip install ipywidgets

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the pre-trained model
model = load_model('/content/drive/MyDrive/images/IMR.h5')

# Define a function to preprocess the input image
def preprocess_image(image_path):
    # Load the image and convert to RGB format
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Resize the image to the input size of the model
    resized = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

    # Reshape the image to match the input shape of the model
    reshaped = np.reshape(resized, (1, 224, 224, 3))

    # Normalize the pixel values to be between 0 and 1
    normalized = reshaped / 255.0

    return normalized

# Define a function to predict the name of the monument
def predict_monument(image_path):
    # Preprocess the input image
    preprocessed_image = preprocess_image(image_path)

    # Make a prediction using the model
    prediction = model.predict(preprocessed_image)

    # Get the index of the predicted class
    predicted_class = np.argmax(prediction)

    # Map the index to the corresponding monument name
    if predicted_class == 0:
        return "Bibi Ka Maqbara - A tomb built in the 17th century in Aurangabad, Maharashtra, India"
    elif predicted_class == 1:
        return "Charminar - A monument and mosque located in Hyderabad, Telangana, India"
    elif predicted_class == 2:
        return "Gol Gumbaz - A mausoleum located in Bijapur, Karnataka, India"
    elif predicted_class == 3:
        return "Hawa Mahal - A palace located in Jaipur, Rajasthan, India"
    elif predicted_class == 4:
        return "Kanch Mahal - A palace located in New Delhi, India"
    elif predicted_class == 5:
        return "Lotus Temple - A Bahá'í House of Worship located in New Delhi, India"
    elif predicted_class == 6:
        return "Parliament - The Parliament of India located in New Delhi, India"
    elif predicted_class == 7:
        return "Red Fort - A historical fort located in Delhi, India"
    elif predicted_class == 8:
        return "Taj Mahal - A mausoleum located in Agra, Uttar Pradesh, India"
    else:
        return "Unknown monument"


# Define a function to prompt the user for an image and return the predicted monument
def predict_from_input():
    # Prompt the user to input the image path
    image_path = input("Hello! Please upload an image of a monument you'd like me to identify: ")

    # Predict the monument name based on the image
    predicted_monument = predict_monument(image_path)

    # Print the predicted monument name
    print("Based on my analysis, this looks like: ", predicted_monument)

    # Ask the user if they want to predict another image
    another_image = input("Do you want to identify another monument? (yes or no): ")

    # If the user wants to predict another image, call this function recursively
    if another_image.lower() == 'yes':
        predict_from_input()
    else:
        print("Thank you for using our monument recognition chatbot")
    
# Call the predict_from_input function to start the chatbot
predict_from_input()

1/1 [==============================] - 1s 1s/step
Based on my analysis, this looks like:  Lotus Temple - A Bahá'í House of Worship located in New Delhi, India
1/1 [==============================] - 0s 37ms/step
Based on my analysis, this looks like:  Taj Mahal - A mausoleum located in Agra, Uttar Pradesh, India


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install flask-ngrok
!pip install pyngrok



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=567d95b45663ea4c180d313480df2812d85e4e96ae0e0f402bb49de0d60910e3
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
!ngrok authtoken 2PQbAG2hWfpylkRtMSoftDxJePU_6dEaVfWfzRd2jx9gDhpU2

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import os
from flask import Flask, request

app = Flask(__name__)

@app.route('/', methods=['GET'])
def index():
    return 'Hello! Please upload an image of a monument you\'d like me to identify.'

@app.route('/predict', methods=['POST'])
def predict():
    # Get the image file from the request
    file = request.files['file']

    # Save the image file
    file_path = 'image.jpg'
    file.save(file_path)

    # Predict the monument name based on the image
    predicted_monument = predict_monument(file_path)

    # Return the predicted monument name
    return predicted_monument

if __name__ == '__main__':
    # Run the app with ngrok
    from pyngrok import ngrok
    ngrok_tunnel = ngrok.connect(5000)
    print('Public URL:', ngrok_tunnel.public_url)

    # Start the Flask app
    app.run(port=5000)


Public URL: https://0c74-35-202-253-165.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from flask import Flask, jsonify, request
import requests
import json
from flask_ngrok import run_with_ngrok

# Load the pre-trained model
model = load_model('/content/drive/MyDrive/images/IMR.h5')

# Define a function to preprocess the input image
def preprocess_image(image_path):
    # Load the image and convert to RGB format
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Resize the image to the input size of the model
    resized = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

    # Reshape the image to match the input shape of the model
    reshaped = np.reshape(resized, (1, 224, 224, 3))

    # Normalize the pixel values to be between 0 and 1
    normalized = reshaped / 255.0

    return normalized

# Define a function to predict the name of the monument
def predict_monument(image_path):
  
    # Preprocess the input image
    preprocessed_image = preprocess_image(image_path)

    # Make a prediction using the model
    prediction = model.predict(preprocessed_image)

    # Get the index of the predicted class
    predicted_class = np.argmax(prediction)

    # Map the index to the corresponding monument name
    if predicted_class == 0:
        return "Bibi Ka Maqbara - A tomb built in the 17th century in Aurangabad, Maharashtra, India"
    elif predicted_class == 1:
        return "Charminar - A monument and mosque located in Hyderabad, Telangana, India"

    else:
        return "Unknown monument"

# Initialize the Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when the app is run

# Define a route for the home page
@app.route('/')
def home():
    return 'Monument recognition chatbot is running!'

# Define a route for predicting the monument from an input image
@app.route('/predict', methods=['POST'])
def predict():
    # Get the image file from the request
    image_file = request.files['image']

    # Save the image file to disk
    image_path = 'input_image.jpg'
    image_file.save(image_path)

    # Predict the monument name based on the image
    predicted_monument = predict_monument(image_path)

    # Return the predicted monument name as a JSON response
    return jsonify({'monument': predicted_monument})

if __name__ == '__main__':
    app.run()  # Run the Flask app



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://2474-35-202-253-165.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [10/May/2023 10:20:18] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/May/2023 10:20:19] "GET /favicon.ico HTTP/1.1" 404 -
